In [53]:
from pathlib import Path
import pandas as pd

ROOT_PATH = Path('.') 
DATA_FILE_PATH = ROOT_PATH.joinpath('videogamesales').joinpath('vgsales.csv')  #videogamesales vgsales.csv netflix-shows netflix_titles.csv
DB_NAME = ROOT_PATH.joinpath('videogame_sales.db')
TABLE_NAME = 'game_sales'


# Read CSV
csv_data = pd.read_csv(DATA_FILE_PATH)

# Convert to DataFrame
df = pd.DataFrame(csv_data)

df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [54]:
import sqlite3
from _sqlite3 import Error


def sql_connection(db_name):
    try:
        db = sqlite3.connect(db_name)
        print("Connection to database has been established")
        return db
    except Error:
        print(Error)

def insert_data(db, data_df):
    data_df.columns = data_df.columns
    data_df.to_sql(TABLE_NAME, con=db, if_exists='replace', index_label='id')

db = sql_connection(DB_NAME)
    
insert_data(db, df)
db.close()

Connection to database has been established


In [92]:
# This query will print the top-10 games for a chosen year

TABLE_NAME = 'game_sales'

def format_money(x):
    return "${:.2f} Mil.".format(x )

def year_query(cur, date_string):
    ex_str = f'SELECT "Name", "Platform", "Genre", "NA_Sales" FROM {TABLE_NAME} WHERE Year LIKE "%{date_string}%" '
    cur.execute(ex_str)
    db_response = cur.fetchall()
    return db_response

def print_db_results(db_response):
    for item in db_response:
        print(item)
    print('')
    print(str(len(db_response)) + " total games found.")

db = sql_connection(DB_NAME)
cur = db.cursor()
cur.execute(f'SELECT min(Year) FROM {TABLE_NAME}')
min_year = int(cur.fetchone()[0])
cur.execute(f'SELECT max(Year) FROM {TABLE_NAME}')
max_year = int(cur.fetchone()[0])

print("This query will print the top-10 games for a chosen year")
print("DB contains data from " + str(min_year) + " to " + str(max_year) + ".")
repeat = 1
while repeat == 1:
    selected_year = input("Enter a year: ")
    if int(selected_year) < min_year or int(selected_year) > max_year:
        print("Invalid year!")
    else:
        repeat = 0
year_db_response = year_query(cur, selected_year)
year_query = pd.DataFrame(year_db_response)
year_query.columns = ["Name", "Platform", "Genre", "N.Am_Sales"]
year_query.sort_values("NA_Sales")
year_query["NA_Sales"] = year_query["NA_Sales"].apply(format_money)
print(year_query.head(10))

db.close()

Connection to database has been established
This query will print the top-10 games for a chosen year
DB contains data from 1980 to 2020.
Enter a year: 1980
              Name Platform     Genre    NA_Sales
0        Asteroids     2600   Shooter  $4.00 Mil.
1  Missile Command     2600   Shooter  $2.56 Mil.
2          Kaboom!     2600      Misc  $1.07 Mil.
3         Defender     2600      Misc  $0.99 Mil.
4           Boxing     2600  Fighting  $0.72 Mil.
5       Ice Hockey     2600    Sports  $0.46 Mil.
6          Freeway     2600    Action  $0.32 Mil.
7           Bridge     2600      Misc  $0.25 Mil.
8         Checkers     2600      Misc  $0.22 Mil.
